In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Pain_Detection/saves/training_data.pkl')

In [ ]:
data = pd.DataFrame(df)

In [ ]:
data.head()

,0,1
0,"[[[46, 32, 33], [52, 32, 29], [52, 32, 29], [5...",0
1,"[[[118, 77, 76], [121, 75, 66], [121, 75, 66],...",0
2,"[[[115, 120, 118], [95, 96, 97], [86, 92, 97],...",0
3,"[[[107, 90, 82], [105, 86, 77], [109, 85, 75],...",1
4,"[[[91, 85, 82], [91, 85, 81], [91, 85, 81], [9...",0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16840 entries, 0 to 16839
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       16840 non-null  object
 1   1       16840 non-null  object
dtypes: object(2)
memory usage: 263.2+ KB


In [ ]:
data.isnull().any()

0    False
1    False
dtype: bool

In [ ]:
data.shape

(16840, 2)

In [ ]:
dataset = data.copy()
dataset.head()

,0,1
0,"[[[46, 32, 33], [52, 32, 29], [52, 32, 29], [5...",0
1,"[[[118, 77, 76], [121, 75, 66], [121, 75, 66],...",0
2,"[[[115, 120, 118], [95, 96, 97], [86, 92, 97],...",0
3,"[[[107, 90, 82], [105, 86, 77], [109, 85, 75],...",1
4,"[[[91, 85, 82], [91, 85, 81], [91, 85, 81], [9...",0


In [ ]:
x = dataset[0]
y = dataset[1]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
import keras as ks
from keras.models import Sequential,Model
from tensorflow.keras.utils import to_categorical
from keras import applications
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
tune_model = applications.vgg16.VGG16(weights='imagenet', include_top=False,input_shape=(200, 200, 3))

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
model = Sequential()

In [ ]:
for layer in tune_model.layers:
    model.add(layer)

for layer in model.layers:
    layer.trainable = False

In [ ]:
model.add(Conv2D(256,3, 3, padding="same",activation='relu'))
model.add(Conv2D(512,3, 3, padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(2,activation='softmax'))

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
x_train.shape
y_train.shape
x_test.shape
y_test.shape

(3368, 2)

In [ ]:
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))
#print(x_train.head())

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
x_train = np.array([np.array(val) for val in x_train])
y_train = np.array([np.array(val) for val in y_train])
x_test = np.array([np.array(val) for val in x_test])
y_test = np.array([np.array(val) for val in y_test])

In [ ]:
print(type(x_train))
print(type(x_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
m = model.fit(x_train, y_train, epochs=20, validation_data = (x_test, y_test),batch_size = 64)

Epoch 1/20
211/211 [==============================] - 83s 309ms/step - loss: 0.6026 - accuracy: 0.7352 - val_loss: 0.4315 - val_accuracy: 0.7850
Epoch 2/20
211/211 [==============================] - 70s 332ms/step - loss: 0.4141 - accuracy: 0.7971 - val_loss: 0.4163 - val_accuracy: 0.7990
Epoch 3/20
211/211 [==============================] - 70s 332ms/step - loss: 0.3937 - accuracy: 0.8051 - val_loss: 0.3879 - val_accuracy: 0.8088
Epoch 4/20
211/211 [==============================] - 70s 331ms/step - loss: 0.3808 - accuracy: 0.8159 - val_loss: 0.3892 - val_accuracy: 0.8067
Epoch 5/20
211/211 [==============================] - 70s 331ms/step - loss: 0.3756 - accuracy: 0.8139 - val_loss: 0.3988 - val_accuracy: 0.8064
Epoch 6/20
211/211 [==============================] - 61s 291ms/step - loss: 0.3710 - accuracy: 0.8189 - val_loss: 0.3935 - val_accuracy: 0.8135
Epoch 7/20
211/211 [==============================] - 68s 325ms/step - loss: 0.3684 - accuracy: 0.8200 - val_loss: 0.3964 - val_ac

In [ ]:
model.save("/content/drive/MyDrive/Pain_Detection/ saves/pain_model.h5")

In [ ]:
print(m.history['loss'])
print(m.history['val_loss'])
print(m.history['accuracy'])
print(m.history['val_accuracy'])


[0.6026208400726318, 0.4141123592853546, 0.3937411606311798, 0.3808440864086151, 0.375591903924942, 0.3709896206855774, 0.3683722913265228, 0.3631714880466461, 0.36021673679351807, 0.3610745966434479, 0.36166685819625854, 0.359640896320343, 0.3537839651107788, 0.3543368875980377, 0.3519669473171234, 0.35362181067466736, 0.35157865285873413, 0.3501318693161011, 0.34823134541511536, 0.34519216418266296]
[0.43148237466812134, 0.41631898283958435, 0.38791462779045105, 0.38916417956352234, 0.3987550437450409, 0.39348727464675903, 0.39638859033584595, 0.3882180154323578, 0.4071435332298279, 0.3920650780200958, 0.4010092616081238, 0.40610164403915405, 0.4120493233203888, 0.3957344591617584, 0.40253835916519165, 0.4098328649997711, 0.42377299070358276, 0.40065646171569824, 0.4053359627723694, 0.4088350832462311]
[0.7352285981178284, 0.7971348166465759, 0.8050771951675415, 0.815914511680603, 0.8139103055000305, 0.8188835978507996, 0.8199970126152039, 0.8241537809371948, 0.8234857320785522, 0.82

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.math_ops import arg_max

In [ ]:
model=tf.keras.models.load_model("/content/drive/MyDrive/Pain_Detection/saves/pain_model.h5")

In [ ]:
import cv2
def test(image):
  img=cv2.imread(image, cv2.IMREAD_COLOR)
  img=cv2.resize(img,(200,200))
  test = np.array(img).reshape(-1, 200, 200, 3)
  prediction=model.predict(test)
  print(prediction)
  print(np.argmax(prediction[0]))
  

In [ ]:
from google.colab.patches import cv2_imshow
image = '/content/drive/MyDrive/Pain_Detection/1.jfif'
#cv2_imshow(image)
test(image)

1/1 [==============================] - 8s 8s/step
[[0.9969126 0.0030874]]
0


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/pain.jfif'
test(image)

1/1 [==============================] - 0s 28ms/step
[[0.63529164 0.3647084 ]]
0


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/nopain.jfif'
test(image)

1/1 [==============================] - 0s 20ms/step
[[0.5549717 0.4450283]]
0


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/2.jfif'
test(image)

1/1 [==============================] - 0s 19ms/step
[[0.593085 0.406915]]
0


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/11.jfif'
test(image)

1/1 [==============================] - 0s 20ms/step
[[0.12307069 0.8769293 ]]
1


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/12.jfif'
test(image)

1/1 [==============================] - 0s 19ms/step
[[0.09390801 0.906092  ]]
1


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/21.jfif'
test(image)

1/1 [==============================] - 0s 20ms/step
[[0.9103476  0.08965246]]
0


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/22.jfif'
test(image)

1/1 [==============================] - 0s 19ms/step
[[0.6845649  0.31543514]]
0


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/31.jfif'
test(image)

1/1 [==============================] - 0s 27ms/step
[[0.4368703 0.5631297]]
1


In [ ]:
image = '/content/drive/MyDrive/Pain_Detection/32.jfif'
test(image)

1/1 [==============================] - 0s 22ms/step
[[0.59169406 0.40830594]]
0
